In [1]:
import numpy as np
import pandas as pd
from main import MLP_Classifier,Layer
from sklearn.datasets import make_classification
import optuna
from sklearn.metrics import accuracy_score,precision_score,recall_score
from sklearn.model_selection import train_test_split

/opt/python/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
np.random.seed(42)
# Generate  dataset
X, Y = make_classification(
    n_samples=1000,     
    n_features=4,       
    n_redundant=0,      
    n_clusters_per_class=1,
    flip_y=0.1,         # Add label noise
    class_sep=1.0,      # Class separation
    n_classes=2,      # nb classes
)


noise = np.random.normal(0, 0.5, X.shape)
X = X + noise
X=pd.DataFrame(X)
Y=pd.Series(Y)
if len(np.unique(Y))==2:
   Y=pd.DataFrame(Y)
else: 
   Y=pd.get_dummies(Y).astype(int)


determine objective function to optimise :  minimise cross entropy (in pdf maximise log-likelihood).

for example we optimise over batch size, learning rate and dropout (one of the most important parameters in NN).

we could do also on layers but computantionally expensive for large datasets.


In [3]:



def objective(trial):
    # Define hyperparameter search space
    batch_size = trial.suggest_int("batch_size", 500, 800)
    alpha = trial.suggest_float("alpha", 0.01, 0.1)
    dropout_rate = trial.suggest_float("dropout", 0.5, 0.9)

    model = MLP_Classifier(
        (
            (
                Layer(
                    nb_neurons=20,
                    activation_function="relu",
                    regul=("l2", 0.1),
                    initial="he",
                    batchnorm=True
                ),
                Layer(
                    nb_neurons=10,
                    activation_function="relu",
                    regul=("l2", 0.1),
                    initial="he",
                ),
                Layer(
                    nb_neurons=30,
                    activation_function="relu",
                    regul=("dropout", dropout_rate),
                    initial="he",
                ),
            )
        ),
        max_iter=2000,
        thr=1e-5,
        alpha=alpha,
        seed=123,
        batch_size=batch_size,
        verbose=False,
        optim="adam"
    )

    model.train(X, Y)

    score = model.loss(Y,model.y_hat)  # need to do on val set

    return score


storage = "sqlite:///optuna_mlpsoftmax.db"
study = optuna.create_study(
    direction="minimize", study_name="MLP", storage=storage, load_if_exists=True
)  # 'minimize' for loss functions
study.optimize(objective, n_trials=1)

print("Best Hyperparameters:", study.best_params)


[I 2025-10-26 14:46:36,265] Using an existing study with name 'MLP' instead of creating a new one.


Don't forget to normalise input data and think about Batch normalisations


[I 2025-10-26 14:46:37,364] Trial 15 finished with value: 0.37788340705543 and parameters: {'batch_size': 700, 'alpha': 0.08220049763763941, 'dropout': 0.746047498058809}. Best is trial 15 with value: 0.37788340705543.


Best Hyperparameters: {'batch_size': 700, 'alpha': 0.08220049763763941, 'dropout': 0.746047498058809}


In [4]:
best_results={"best value" : study.best_trial.values,"params": study.best_trial.params}
best_results


{'best value': [0.37788340705543],
 'params': {'batch_size': 700,
  'alpha': 0.08220049763763941,
  'dropout': 0.746047498058809}}

run model on optimised parameters

In [5]:


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

model = MLP_Classifier(
    (
        (
            Layer(
                nb_neurons=20,
                activation_function="relu",
                regul=("l2", 0.1),
                initial="he",
                batchnorm=True
         
            ),
            Layer(
                nb_neurons=10,
                activation_function="relu",
                regul=("l2", 0.1),
                initial="he",
                
            ),
            Layer(
                nb_neurons=30,
                activation_function="relu",
                regul=("dropout", best_results["params"]["dropout"]),
                initial="he",
                
            ),
        )
    ),
    max_iter=2000,
    thr=1e-5,
    alpha=best_results["params"]["alpha"],
    seed=123,
    batch_size=best_results["params"]["batch_size"],
    verbose=True,
    optim="adam",
    nb_epochs_early_stopping=50
)

fct=accuracy_score

model.train(X_train, y_train,X_test,y_test,fct)

print(f"final {fct.__name__}", accuracy_score(model.predict(X_test), y_test))


-------------------------------------------------------------------------
iteration 0 : TRAIN accuracy_score  : 0.748, loss : 0.5357972415572081
iteration 0 : TEST accuracy_score  : 0.716, loss : 0.5455762295857955
-------------------------------------------------------------------------
iteration 50 : TRAIN accuracy_score  : 0.8293333333333334, loss : 0.3997065067777386
iteration 50 : TEST accuracy_score  : 0.864, loss : 0.3534494506856128
early stopping at epoch 79
final accuracy_score 0.864


In [1]:
from layers import ConvLayer,MaxPoolLayer,Layer,FlatLayer
from cnn import CNN
import torch

In [2]:

q=CNN(

    (
        ConvLayer(in_channels=3,output_channels=16,kernel_size=2,stride=1,padding=True,activation_function="relu",regul=None,initial="lecun",law="normal",batchnorm=False),
        MaxPoolLayer(kernel_size=3,stride=2,padding=True),
        ConvLayer(in_channels=16,output_channels=32,kernel_size=2,stride=1,padding=True,activation_function="relu",regul=None,initial="lecun",law="normal",batchnorm=False),
        MaxPoolLayer(kernel_size=3,stride=2,padding=True),
        ConvLayer(in_channels=32,output_channels=64,kernel_size=2,stride=1,padding=True,activation_function="relu",regul=None,initial="lecun",law="normal",batchnorm=False),
        MaxPoolLayer(kernel_size=3,stride=2,padding=True),
        FlatLayer(),
        Layer(
                nb_neurons=20,
                activation_function="relu",
                regul=("l2", 0.1),
                initial="he",
                batchnorm=True
         
            ),
            Layer(
                nb_neurons=10,
                activation_function="relu",
                regul=("l2", 0.1),
                initial="he",
                
            ),
            Layer(
                nb_neurons=30,
                activation_function="relu",
                regul=("dropout",0.5),
                initial="he",
                
            ),
        
    )



)

Don't forget to normalise input data and think about Batch normalisations


In [3]:
q.conv_kernel={}
q.conv_bias={}
q.kernels_init()

In [4]:
q.conv_kernel[3 ]

tensor([[[[-0.1225, -0.2641],
          [-0.0340, -0.0439]],

         [[ 0.1394, -0.0772],
          [-0.2839, -0.1727]],

         [[ 0.1465, -0.0546],
          [-0.0507,  0.0886]],

         ...,

         [[ 0.2322,  0.2665],
          [-0.0632, -0.0999]],

         [[-0.1368, -0.1275],
          [-0.0675,  0.1515]],

         [[-0.1079,  0.1667],
          [ 0.0096, -0.0065]]],


        [[[ 0.0298,  0.0176],
          [-0.1669, -0.3668]],

         [[ 0.0143, -0.1509],
          [-0.0376,  0.0178]],

         [[-0.1628, -0.0615],
          [-0.2679,  0.1186]],

         ...,

         [[-0.0038, -0.0112],
          [-0.0246, -0.1214]],

         [[ 0.1126, -0.0315],
          [ 0.1334, -0.0373]],

         [[ 0.1070,  0.2012],
          [-0.1487,  0.1460]]],


        [[[ 0.0410, -0.1041],
          [-0.2022,  0.0283]],

         [[-0.0548,  0.0408],
          [-0.1973, -0.1749]],

         [[ 0.0681, -0.0104],
          [-0.1469,  0.2228]],

         ...,

         [[ 0.0177,  

In [28]:
q.network[1]["layer_type"].__name__

'ConvLayer'

In [40]:
import numpy as np

#in mlp netwrok:
fan_in=2
fan_out=3

np.random.normal(0, np.sqrt(1 / fan_in), (fan_in, fan_out))

#in cnn let kernel be shape (16,3,2,2)
fan_in=2*2*3
fan_out=2*2*16

np.random.normal(0, np.sqrt(1 / fan_in), (16, 3,2,2))


array([[[[-3.05320347e-01,  2.73165973e-02],
         [ 1.75627046e-02, -8.15219774e-02]],

        [[-2.62418893e-01, -1.17711239e-01],
         [ 2.54002437e-02, -3.72959268e-01]],

        [[-3.07543311e-01, -3.90278487e-01],
         [-2.28766389e-01,  7.96790175e-02]]],


       [[[-2.20186808e-01, -2.01998420e-01],
         [ 5.18998067e-01,  1.44602622e-01]],

        [[ 3.42836532e-01, -2.49860842e-01],
         [ 1.91245650e-01,  1.96779655e-01]],

        [[-3.48306552e-02, -1.49213731e-01],
         [-7.24786023e-02,  1.38582444e-01]]],


       [[[-5.36575490e-01, -3.85727115e-02],
         [-3.71185928e-01, -4.10202974e-01]],

        [[-2.87387085e-01, -6.39429425e-01],
         [-3.61344098e-02, -1.22941042e-03]],

        [[-1.03917460e-01,  1.94367237e-01],
         [ 2.19478507e-01,  1.55339075e-01]]],


       [[[ 3.69886620e-01, -2.72440569e-01],
         [-1.07244907e-01,  4.38595579e-01]],

        [[-1.42125179e-02,  5.79476579e-01],
         [ 1.84028196e-01, -2

In [ ]:
q.nb_cnn_layers+q.nb_layers#total

from q.nb_cnn_layers+1 to total+1 do gradients mlp 


from 1 to q.nb_cnn_layers+1 do gradients cnn 

8

In [35]:

conv = nn.Conv2d(in_channels=3,out_channels=16,kernel_size=2)
print(conv.weight.shape)
torch.Size([1, 1, 2, 2])

print(nn.init._calculate_fan_in_and_fan_out(conv.weight))


torch.Size([16, 3, 2, 2])
(12, 64)


In [19]:
fanin=3*3*3
FANOUT=3*3*16
fanin,FANOUT

(27, 144)

In [30]:
from torch import nn